In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [48]:
df = pd.read_csv('imdb_master.csv', encoding = "ISO-8859-1")
del df['Unnamed: 0']
del df['file']
del df['type']

In [49]:
df = df.loc[0:49999]
df = df.sample(frac=1).reset_index(drop=True)

In [50]:
def clean_reviews(no_reviews,dataset):
    import re
    import nltk
    nltk.download('stopwords') 
    from nltk.corpus import stopwords 
    
    my_stopwords = stopwords.words('english') 
            
    from nltk.stem.porter import PorterStemmer
     
    corpus =[] 
    for i in range (0, no_reviews):
        review = re.sub('[^a-zA-Z]',' ', dataset['review'][i]) #Removes all other characters besides the letters and puts the review in a string
        review = review.lower() 
        review = review.split()
        ps = PorterStemmer() 
        review = [ps.stem(word) for word in review if not word in set (my_stopwords)]
        review = ' '.join(review) 
        corpus.append(review)
    return corpus

In [51]:
x = clean_reviews(50000,df)

[nltk_data] Downloading package stopwords to /home/aayush/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
def tf_idf(corpus):
    from sklearn.feature_extraction.text import TfidfVectorizer
    vect = TfidfVectorizer(max_features = 5000)
    X = vect.fit_transform(corpus)
    return X

In [53]:
x_data = tf_idf(x)

In [54]:
x_data

<50000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 4160593 stored elements in Compressed Sparse Row format>

In [55]:
labels = df['label']
labels = labels.map({'pos': 1, 'neg': 0})

In [56]:
train_data = x_data[0:34999,:]
test_data = x_data[35000:49999,:]
train_labels = labels[0:34999]
test_labels = labels[35000:49999]

In [57]:
from sklearn.svm import LinearSVC
svm = LinearSVC()

In [58]:
svm.fit(train_data, train_labels)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [59]:
y_pred_class = svm.predict(test_data)

In [60]:
from sklearn import metrics
metrics.accuracy_score(test_labels, y_pred_class)

0.8813920928061871

In [61]:
metrics.confusion_matrix(test_labels, y_pred_class)

array([[6561,  933],
       [ 846, 6659]])